In [2]:
import pandas as pd
import numpy as np


In [5]:


from sqlalchemy import create_engine



def initializeDB():
    # Load environment variables from .env file


    # Retrieve database connection parameters from the .env file
    db_params = {
        'host': 'localhost',
        'port': 7500,
        'dbname': 'orchestro',
        'user': 'analytics_user',
        'password': 'g2mKunhe4ZTSeSQGni8e'
    }

    # Creating the database URI using the db_params dictionary
    db_uri = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

    # Create an SQLAlchemy engine
    engine = create_engine(db_uri)
   
    return engine

engine = initializeDB()

In [6]:
# test the db connection

query = """
SELECT
    origin_carrier, destination_carrier,
    COUNT(*) AS count,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ()), 2) AS percentage
FROM
    analytics_scrap.easypost
WHERE
    origin_carrier IS NOT NULL AND destination_carrier IS NOT NULL
    AND origin_carrier != 'EXT' AND destination_carrier != 'EXT'
GROUP BY
    origin_carrier, destination_carrier
ORDER BY
    count DESC
"""

carrier_df = pd.read_sql(query, engine)
carrier_df.head(3)

,origin_carrier,destination_carrier,count,percentage
0,GLS,GLS,280143,14.24
1,LSO,LSO,147577,7.50
2,GLS,LSO,126647,6.44


In [7]:
# load the cleaned up data from Deeksha
query = """
SELECT
    *
FROM
    analytics_scrap.cleaned_events

"""

df_events_clean = pd.read_sql(query, engine)
df_events_clean.shape

(8422638, 7)

In [8]:
# sort the event data 
df_events = df_events_clean.sort_values(['tracking_number', 'min_timestamp'])
#df_events.head(3)

# Count the occurrences of each tracking_number
tracking_counts = df_events['tracking_number'].value_counts()
tracking_counts.shape

# pick tracking numbers that have a reasonable amount of data
good_id = tracking_counts.loc[tracking_counts > 10]
good_id_list = list(good_id.index)

df_events_small = df_events[df_events['tracking_number'].isin(good_id_list) ]
df_events_small.head(10)

,tracking_number,carrier_name,status,city,state,zip,min_timestamp
4416929,BU0000E4,LSO,ISD,AUSTIN,TX,78753,2023-12-21T04:33:00-06:00
4439180,BU0000E4,LSO,P,AUSTIN,TX,78753,2023-12-21T10:25:58-06:00
4443051,BU0000E4,LSO,DPS,PLANO,TX,75074,2023-12-21T11:40:00-06:00
4443092,BU0000E4,LSO,V,PLANO,TX,75074,2023-12-21T11:40:44-06:00
4443329,BU0000E4,LSO,P,DALLAS,TX,75247,2023-12-21T11:49:00-06:00
4443333,BU0000E4,LSO,P,GRAND PRAIRIE,TX,75050,2023-12-21T11:49:00-06:00
4716585,BU0000E4,LSO,DPS,AUSTIN,TX,78753,2023-12-27T08:00:00-06:00
4716665,BU0000E4,LSO,V,AUSTIN,TX,78753,2023-12-27T08:00:34-06:00
4789023,BU0000E4,LSO,UTL,AUSTIN,TX,78753,2023-12-27T16:03:26-06:00
4860867,BU0000E4,LSO,D,AUSTIN,TX,78753,2023-12-28T09:02:12-06:00


In [9]:
df_events_small.shape

(5668, 7)

In [12]:
# Sort DataFrame by timestamp
df = df_events_small
# treat timestamp as date time
df['min_timestamp'] = pd.to_datetime(df['min_timestamp'])


df_sorted = df.sort_values(by = ['tracking_number', 'min_timestamp' ])

# Create empty columns for status_in and status_out
df_sorted['status_in'] = None
df_sorted['zip_in'] = None
df_sorted['time_in_transit'] = None

# Iterate through consecutive rows with the same package ID
prev_row = None
for index, row in df_sorted.iterrows():
    if prev_row is not None and prev_row['tracking_number'] == row['tracking_number']:
        df_sorted.at[index, 'status_in'] = prev_row['status']
        df_sorted.at[row.name, 'zip_in'] = prev_row['zip']
        df_sorted.at[row.name, 'time_in_transit'] = (row['min_timestamp'] - prev_row['min_timestamp']).total_seconds()/3600
        
    prev_row = row

# Drop rows where status_in or status_out is None (for first or last row of each package ID)
#df_sorted.dropna(subset=['status_in', 'status_out'], inplace=True)


# Reset index
#df_sorted.reset_index(drop=True, inplace=True)

# print shape of df_sorted
#print('df_sorted shape is ', df_sorted.shape)

# drop first row with None
df_sorted = df_sorted.iloc[1:, ]
df_sorted.tail(3)

# drop null values for time 
df_sorted.dropna(subset=['time_in_transit'], inplace=True)

/var/folders/lh/bbmk74hd62nfqmlc2kyrns300000gn/T/ipykernel_9381/2956933292.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['min_timestamp'] = pd.to_datetime(df['min_timestamp'])


In [13]:
# Compute median of time_in_transit_hours for each combination of status_in and status
median_transit_time = df_sorted.groupby(['status_in', 'status'])['time_in_transit'].median().reset_index()

# Rename the column to reflect the median transit time
median_transit_time.rename(columns={'time_in_transit': 'median_transit_time'}, inplace=True)

# Display the resulting DataFrame
print(median_transit_time)

    status_in status  median_transit_time
0         ADS    ADS            26.345000
1         ADS    BDA            22.956667
2         ADS    BDR            17.811389
3         ADS    CLO            65.391806
4         ADS      D            30.431250
..        ...    ...                  ...
316       int    INT            26.413056
317       int    ISD            15.794167
318       int    LBX            74.522500
319       mis    BDA            25.084722
320       mis    MIS            98.522222

[321 rows x 3 columns]


In [15]:
# Compute median and count for each combination of status_in and status
df = df_sorted

# Compute median and count for each combination of status_in and status
result = df.groupby(['status_in', 'status']).agg(median_transit_time=('time_in_transit', 'median'), count=('time_in_transit', 'count')).reset_index()

# Display the resulting DataFrame
#print(result)

df = result[result['count'] > 20 ].sort_values(by = ['count'])
df.shape

(44, 4)

In [23]:
# Example DataFrame with status_in, status, median_transit_time, and count
# data = {
#     'status_in': ['A', 'A', 'A', 'B', 'B', 'C'],
#     'status': ['X', 'Y', 'Z', 'Y', 'Z', 'X'],
#     'median_transit_time': [10, 15, 20, 12, 18, 8],
#     'count': [5, 3, 4, 7, 6, 2]
# }
# df = pd.DataFrame(data)

global df, df_t

df = result[result['count'] > 20 ].sort_values(by = ['count'])

df_t = df

# Function to find all possible paths between status_in and status
def find_paths(status_in, status, visited, path, paths):
    # Mark the current status as visited
    visited[status_in] = True
    # Append the current status to the path
    path.append(status_in)
    
    # Base case: if we reach the destination status, add the current path to the list of paths
    if status_in == status:
        paths.append(path.copy())
    else:
        # Explore all possible next statuses from the current status
        for next_status in df[df['status_in'] == status_in]['status']:
            if not visited[next_status]:
                find_paths(next_status, status, visited, path, paths)
    
    # Backtrack: unmark the current status as visited and remove it from the path
    visited[status_in] = False
    path.pop()

# Function to compute weighted transit time for a given path
def compute_weighted_transit_time(path):
    weighted_transit_time = 0
    factor = 1
    total_count = 1  # Start with 1 package
    for i in range(len(path) - 1):
        status_in = path[i]
        status = path[i + 1]
        # Find the row corresponding to the current status_in and status
        row = df[(df['status_in'] == status_in) & (df['status'] == status)].iloc[0]
        prob_factor = row['count']/df_t[df_t['status_in'] == status_in]['count'].sum()
        prob_factor = prob_factor * factor
        #print('prob factor is ', prob_factor)
        # Accumulate the weighted transit time along the path
        weighted_transit_time += row['median_transit_time'] * prob_factor
        #print(weighted_transit_time)
        factor = prob_factor
    return weighted_transit_time

# Main function to compute all possible paths and their weighted transit times
def compute_all_paths(status_in, status):
    # Initialize visited array to keep track of visited statuses
    visited = {status: False for status in set(df['status_in']) | set(df['status'])}
    paths = []
    all_paths = []
    find_paths(status_in, status, visited, [], paths)
    
    # make a smaller data frame with the relevant paths
    in_statuses = []
    out_statuses = []
    for path in all_paths:
        list_of_statuses = path[0]
        in_statuses.append(path[0][0: len(path[0])-1])
        out_statuses.append(path[0][1: len(path[0])])
    in_s = {x for l in in_statuses for x in l}
    out_s = {x for l in out_statuses for x in l}
    
    df_t = df[df['status_in'].isin(in_s)]
    df_t = df_t[df_t['status'].isin(out_s)]
    #print(df_t.shape)
    
    # Compute weighted transit time for each path
    result = []
    for path in paths:
        weighted_transit_time = compute_weighted_transit_time(path)
        result.append((path, weighted_transit_time))
    return result

# Example usage
#status_in = 'B'
#status = 'F'

status_in = 'DPS'
status = 'D'

time_in_transit = 0
all_paths = compute_all_paths(status_in, status)
print("All possible paths between status_in {} and status {}: ".format(status_in, status))
for path, weighted_transit_time in all_paths:
    print("Path:", "->".join(path), "Weighted Transit Time:", weighted_transit_time)
    time_in_transit += weighted_transit_time

print("\n \n \n")
print("The TNT (time in transit) between status_in {} and status{}  ".format(status_in, status), " is ", time_in_transit)

All possible paths between status_in DPS and status D: 
Path: DPS->BDR->V->ISD->UTL->D Weighted Transit Time: 0.016513216361538
Path: DPS->BDR->V->ISD->D Weighted Transit Time: 0.015333266279072123
Path: DPS->BDR->V->ISD->ADS->D Weighted Transit Time: 0.015269906970798778
Path: DPS->BDR->V->ISD->ADS->INT->FDR->D Weighted Transit Time: 0.014925160414020769
Path: DPS->BDR->V->ISD->ADS->INT->D Weighted Transit Time: 0.015037838537892116
Path: DPS->BDR->V->ISD->HPU->INT->FDR->D Weighted Transit Time: 0.016771131399740066
Path: DPS->BDR->V->ISD->HPU->INT->D Weighted Transit Time: 0.017233306208144828
Path: DPS->BDR->V->ISD->HPU->ADS->D Weighted Transit Time: 0.016951435087260124
Path: DPS->BDR->V->ISD->HPU->ADS->INT->FDR->D Weighted Transit Time: 0.016361388095851605
Path: DPS->BDR->V->ISD->HPU->ADS->INT->D Weighted Transit Time: 0.01655424103863141
Path: DPS->BDR->V->FDR->D Weighted Transit Time: 0.02200144959958735
Path: DPS->BDR->V->D Weighted Transit Time: 0.03787136750434346
Path: DPS-

In [18]:
df

,status_in,status,median_transit_time,count
278,UTL,ISD,32.801667,21
257,UTA,ISD,49.197222,21
19,BDA,D,4.672222,21
247,UTA,BDA,21.194444,23
4,ADS,D,30.431250,24
192,ISD,UTL,16.354583,24
193,ISD,V,8.387591,25
177,ISD,D,16.500000,25
173,ISD,ADS,6.045278,27
22,BDA,HPU,9.704583,28


In [21]:
compute_all_paths('DPS', 'D')

NameError: name 'all_paths' is not defined